In [7]:
import pandas as pd
business_json_path = 'data/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)

# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1]

drop_columns = ['hours','is_open','review_count']
df_b = df_b.drop(drop_columns, axis=1)

business_RV = df_b[df_b['categories'].str.contains(
              'RV Repair|RV Dealers|RV Rental|RV Parks|Campgrounds',
              case=False, na=False)]

df_explode = df_b.assign(categories = df_b.categories
                         .str.split(', ')).explode('categories')

df_explode.categories.value_counts()

df_explode[df_explode.categories.str.contains('RV',
                      case=True,na=False)].categories.value_counts()

review_json_path = 'data/yelp_academic_dataset_review.json'
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)


chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['review_id','useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_RV, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

csv_name = "yelp_reviews_RV_categories.csv"
df.to_csv(csv_name, index=False)




1114 out of 1,000,000 related reviews
1124 out of 1,000,000 related reviews
999 out of 1,000,000 related reviews
1065 out of 1,000,000 related reviews
848 out of 1,000,000 related reviews
993 out of 1,000,000 related reviews
1831 out of 1,000,000 related reviews


In [18]:
data = pd.read_csv("./yelp_reviews_RV_categories.csv")
data.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,attributes,categories,user_id,review_stars,text,date
0,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik...",0VMuCPgwZliInxGWfJnxKQ,5,"We set up an online appt, meeting and transact...",2016-07-04 00:23:48
1,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik...",OQLafElx2LwCUM9kMgia5g,5,This was my first ATV ride ever and it was ama...,2016-05-31 16:02:28
2,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik...",jNIjvANki1UpT-WYbYRoWA,5,This is the place to go if you want to experie...,2015-12-05 04:36:35
3,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik...",2-RkiyLH35BVFYwyH7Su9g,5,"Had a wonderful time with Nick, our guide! He ...",2016-07-06 04:44:56
4,25Uww0C0wvF9CZ_3B6vWtA,Enjoy The Mountain,1 Garden St,Santa Barbara,CA,93105,34.414342,-119.687332,4.5,"{'WheelchairAccessible': 'True', 'BikeParking'...","Motorcycle Rental, Tours, Hiking, Mountain Bik...",Camily8WGaay74gKCod0CQ,5,"Nick really took care of us, my boy and I, on ...",2016-08-15 14:51:48


In [5]:
pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached regex-2024.4.16-cp312-cp312-win_amd64.whl.metadata (41 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached regex-2024.4.16-cp312-cp312-win_amd64.whl (268 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\nltk.exe' -> 'c:\\Python312\\Scripts\\nltk.exe.deleteme'



In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Leo
[nltk_data]     Hirano\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords

# Load your data
data = pd.read_csv('yelp_reviews_RV_categories.csv')

# Replace NaN values in the 'text' column with ""
data['text'].fillna("", inplace=True)

# Replace NaN values in the 'attributes' column with a default label or ""
data['attributes'].fillna("unknown", inplace=True)  # you can replace "unknown" with any default label or ""

texts = data['text']
labels = data['attributes']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Creating a pipeline that first creates bag of words (after removing stopwords), then applies TF-IDF
pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words=stopwords.words('english'))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

# Train the classifier
pipeline.fit(X_train, y_train)



C:\Users\Leo Hirano\AppData\Local\Temp\ipykernel_13776\2336857413.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['text'].fillna("", inplace=True)
C:\Users\Leo Hirano\AppData\Local\Temp\ipykernel_13776\2336857413.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

Pipeline(steps=[('vect',
                 CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [17]:
# Make predictions
y_pred = pipeline.predict(X_test)

# Print out the classification report and accuracy
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  precision    recall  f1-score   support

                                             

c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
